In [ ]:
#works with hpatches conda environment

In [3]:
import sys
import glob
import os
import cv2
import numpy as np

from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D, Conv2D, Flatten
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras import initializers
import keras

In [8]:
folder_path = '/scratch/cloned_repositories/hpatches-benchmark/data/hpatches-release/'
patch_size = 65

input_patch_size = 56
brief_patch_size = 48

In [9]:
# all types of patches 
tps = ['ref','e1','e2','e3','e4','e5','h1','h2','h3','h4','h5',\
       't1','t2','t3','t4','t5']

class hpatches_sequence:
    """Class for loading an HPatches sequence from a sequence folder"""
    itr = tps
    def __init__(self,base):
        name = base.split('/')
        self.name = name[-1]
        self.base = base
        for t in self.itr:
            im_path = os.path.join(base, t+'.png')
            im = cv2.imread(im_path,0)
            self.N = im.shape[0]/patch_size
            setattr(self, t, np.split(im, self.N))

In [10]:
seqs = glob.glob(folder_path+'/*')
seqs = [os.path.abspath(p) for p in seqs]

In [11]:
len(seqs)

116

In [18]:
descr_name = 'learned_brief_64_binary_datagen_100more'
base_dir = '/scratch/image_datasets/2_for_learned_brief/ready'
model_version = '0.2.0.0.4_whole_datagen_dense1152_conv32_conv32_up_conv32_conv32_up_conv32_conv8_conv1__relu_last_sigmoid_bce_output24X24_increasedgeometricnoise_train50more'

In [19]:
brief_ae = load_model(base_dir + '/brief_ae_' + model_version + '.h5')

2022-10-18 22:23:36.782521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 22:23:36.785858: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [7]:
# for keras brief
# averaging_kernel_weight_3x3 = 1./9
# weights_dense = np.load("/home/niaki/PycharmProjects/learned-brief/weights/weights_dense_64.npy")

In [20]:
input_shape = (input_patch_size, input_patch_size, 1)
input_img = Input(shape=input_shape)

x = Conv2D(1, (3,3), activation="relu", padding="valid")(input_img)
x = Conv2D(1, (3, 3), activation="relu", padding="valid")(x)
x = Conv2D(1, (3, 3), activation="relu", padding="valid")(x)
average_blurred = Conv2D(1, (3, 3), activation="relu", padding="valid")(x)
flattened = Flatten(data_format="channels_last")(average_blurred)
briefed = Dense(512, activation="sigmoid")(flattened)

brief_encoder = Model(input_img, briefed)

for i in range(len(brief_encoder.layers)):
    brief_encoder.get_layer(index=i).set_weights(brief_ae.get_layer(index=i).get_weights())

brief_encoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 56, 56, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 54, 54, 1)         10        
                                                                 
 conv2d_1 (Conv2D)           (None, 52, 52, 1)         10        
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 50, 1)         10        
                                                                 
 conv2d_3 (Conv2D)           (None, 48, 48, 1)         10        
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 512)               118016

In [21]:
# output shape, and flattened shape
print(brief_encoder.output_shape[1:])
print(np.prod(brief_encoder.output_shape[1:]))

(512,)
512


In [ ]:
# for seq_path in seqs:
#     seq = hpatches_sequence(seq_path)
#     path = os.path.join(descr_name,seq.name)
#     if not os.path.exists(path):
#         os.makedirs(path)
#     descr = np.zeros((seq.N,2)) # trivial (mi,sigma) descriptor
#     for tp in tps:
#         print(seq.name+'/'+tp)
#         for i,patch in enumerate(getattr(seq, tp)):
#             mi = np.mean(patch) # trivial (mi,sigma) descriptor
#             sigma = np.std(patch) # trivial (mi,sigma) descriptor
#             descr[i] = np.array([mi,sigma])
#         np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',') # X is an array

In [ ]:
for seq_path in seqs:
    seq = hpatches_sequence(seq_path)
    path = os.path.join(descr_name,seq.name)
    if not os.path.exists(path):
        os.makedirs(path)

    descr_size_per_patch = np.prod(brief_encoder.output_shape[1:])
    descr = np.zeros((int(seq.N),descr_size_per_patch))
    
    for tp in tps:
        print(seq.name+'/'+tp)
        for i,patch in enumerate(getattr(seq, tp)):
            patch = patch / 255.0
#             print(patch)
            patch_crop = patch[4: 60, 4: 60]
            patch_crop_encoded = brief_encoder.predict(np.expand_dims(np.expand_dims(patch_crop, axis=-1), axis=0))
            patch_crop_encoded_flat = patch_crop_encoded.flatten()
            
            patch_crop_encoded_flat_binary = patch_crop_encoded_flat.round().astype(int)
#             print(patch_crop_encoded_flat_binary.shape)
#             print(type(patch_crop_encoded_flat_binary[0]))
#             print(patch_crop_encoded_flat_binary)
#             print()
            descr[i] = patch_crop_encoded_flat_binary
        np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',')

In [ ]:
descr_name = 'opencv_brief'
brief = cv2.xfeatures2d.BriefDescriptorExtractor_create(64, False)
keypoint = cv2.KeyPoint((patch_size - 1) / 2, (patch_size - 1) / 2, _size=brief_patch_size)
keypoints = [keypoint]

In [ ]:
for seq_path in seqs:
    seq = hpatches_sequence(seq_path)
    path = os.path.join(descr_name,seq.name)
    if not os.path.exists(path):
        os.makedirs(path)

    descr_size_per_patch = 512
    descr = np.zeros((int(seq.N),descr_size_per_patch))
    
    for tp in tps:
        print(seq.name+'/'+tp)
        for i,patch in enumerate(getattr(seq, tp)):
                        
            _, descriptors = brief.compute(patch, keypoints)
            patch_descr = descriptors[0]
            
            patch_descr = np.unpackbits(patch_descr)
            
#             print(patch_descr.shape)
#             print(type(patch_descr[0]))
#             print(patch_descr)
#             print()
            descr[i] = patch_descr
        np.savetxt(os.path.join(path,tp+'.csv'), descr, delimiter=',')